In [12]:
import pandas as pd
import configparser
import os
import logging
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType
from pyspark.sql.functions import udf, col, lit, year, month, upper, to_date
from pyspark.sql.functions import monotonically_increasing_id

In [2]:
import os
import glob
all_files = []
filepath="../../data/18-83510-I94-Data-2016"
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*.sas7bdat'))
    for f in files :
        all_files.append(os.path.abspath(f))
all_files.remove('/data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat')
print(all_files)

['/data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat', '/data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat']


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

for file in all_files:
    df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../..'+file)
    if file =='/data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat':
        data_set=df_spark
    else:
        data_set=df_spark.union(data_set)

In [4]:
def rename_columns(table, new_columns):
    for original, new in zip(table.columns, new_columns):
        table = table.withColumnRenamed(original, new)
    return table

In [7]:
cols = ("insnum", "entdepu", "occup","visapost")
data_set.drop(*cols).printSchema()


root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [10]:
data_set.na.drop()

DataFrame[cicid: double, i94yr: double, i94mon: double, i94cit: double, i94res: double, i94port: string, arrdate: double, i94mode: double, i94addr: string, depdate: double, i94bir: double, i94visa: double, count: double, dtadfile: string, visapost: string, occup: string, entdepa: string, entdepd: string, entdepu: string, matflag: string, biryear: double, dtaddto: string, gender: string, insnum: string, airline: string, admnum: double, fltno: string, visatype: string]

In [15]:
fact_immr = data_set.select('cicid', 'i94yr', 'i94mon', 'i94port', 'i94addr','arrdate', 'depdate', 'i94mode', 'i94visa').distinct()\
.withColumn("immigration_id", monotonically_increasing_id())

new_columns = ['cic_id', 'year', 'month', 'city_code', 'state_code','arrive_date', 'departure_date', 'mode', 'visa']
fact_immr = rename_columns(fact_immr, new_columns)

In [16]:
fact_immr.printSchema()

root
 |-- cic_id: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- arrive_date: double (nullable = true)
 |-- departure_date: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- visa: double (nullable = true)
 |-- immigration_id: long (nullable = false)



In [17]:
def SAS_to_date(date):
    if date is not None:
        return pd.to_timedelta(date, unit='D') + pd.Timestamp('1960-1-1')
SAS_to_date_udf = udf(SAS_to_date, DateType())


In [18]:
fact_immr=fact_immr.withColumn("country",lit("United States"))
fact_immr = fact_immr.withColumn('arrive_date',SAS_to_date_udf(col('arrive_date')))
fact_immr = fact_immr.withColumn('departure_date',SAS_to_date_udf(col('departure_date')))
fact_immr.printSchema()

root
 |-- cic_id: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- arrive_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- mode: double (nullable = true)
 |-- visa: double (nullable = true)
 |-- immigration_id: long (nullable = false)
 |-- country: string (nullable = false)



In [19]:
with open('./I94_SAS_Labels_Descriptions.SAS') as r:
    content=r.readlines()
content=[s.strip() for s in content]
ports=content[302:962]
data=[port.split('=') for port in ports]
port_code=[x[0].replace("'","").strip() for x in data]
both=[x[1].replace("'","").strip() for x in data]
cities_code= [x.split(",")[0] for x in both]
staties_code=[x.split(",")[-1] for x in both]

data_location=pd.DataFrame({'port_code': port_code,'cities_code':cities_code,'staties_code':staties_code})
data_location.head()

,port_code,cities_code,staties_code
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


In [21]:
wrong_ports=data_location[data_location['cities_code'] == data_location['staties_code']]
set_ports=list(set(wrong_ports['port_code'].values))
print(set_ports)

['ZZZ', 'WTR', 'NYL', 'CPX', 'GPI', 'CLX', 'SP0', 'A2A', 'W55', 'OAI', 'WA5', 'XXX', 'X44', 'SCH', 'CXO', 'OGS', 'OSN', 'CP', 'MAA', 'NK', 'BCM', 'LIT', 'AG', '74S', 'MTH', 'FRG', '888', 'WAS', 'OTS', 'GAC', 'OLM', 'IAG', 'ATW', 'CHN', 'Y62', 'DRV', 'SUS', 'FTB', 'ASI', 'JSJ', 'NC8', 'STN', 'RYY', 'ISP', '060', 'BKF', 'PFN', 'PLB', 'T01', 'JIG', 'NGL', 'PHF', 'UNK', 'AMT', 'ADU', 'JBQ', 'PHN', 'HRL', 'AUH', 'JFA', 'GMT', '.GA', '5T6', 'EGE', 'AKT', 'DEC', 'YGF', 'FSC', 'VMB', 'PCW', 'X96', 'XNA', 'BUS', 'TIW', 'DAY', 'JMZ', 'MAP']


In [22]:
fact_immr_table=fact_immr[~fact_immr['city_code'].isin(set_ports)]